<a href="https://colab.research.google.com/github/GajananKuchgave/EVA8_Assignments/blob/main/MNIST_2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Neural Network to detect MNIST data and Sum of MNIST number + Random Number

#Libraries

In [1]:
#Import required libraries
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader

import torch.nn.functional as F
import torch.optim as optim


import matplotlib.pyplot as plt
import numpy as np
import random

#Use Cuda as a device Type

In [2]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


#Download the Training and Testing Dataset

In [3]:
# Train set
MNIST_train_set = torchvision.datasets.MNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences
    ])
)

print("Training set size",len(MNIST_train_set))

MNIST_test_set = torchvision.datasets.MNIST(
    root='./data'
    ,train=False
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences
    ])
)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Training set size 60000


#Customized dataset to read both random number and MNIST data

In [4]:
class MNISTandRandomDataset(Dataset):
  def __init__(self, MNIST_dataset):
    self.MNIST_dataset = MNIST_dataset

  def __getitem__(self, index):
    image, label = self.MNIST_dataset[index]
    randNum = random.randint(0,9)

 #Creating one hot encoding for random number 
    one_hot_randNumber = F.one_hot(torch.arange(0, 10)) 
    #add actual label and random number
    sum_label = label + randNum
    return image, label, one_hot_randNumber[randNum], sum_label

  def __len__(self):
    return len(self.MNIST_dataset)

#Training and Testing Dataset

In [5]:

train_data_set = MNISTandRandomDataset(MNIST_train_set)
test_data_set = MNISTandRandomDataset(MNIST_test_set)

#Dataloader for both batch based training and Testing


In [6]:
train_loader = torch.utils.data.DataLoader(train_data_set, batch_size = 128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data_set,batch_size=128)

#Network Class for MNIST and Random NN Model

In [21]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

        self.fc1 = nn.Linear(20, 64)    
        # Total of 19 outputs for MNIST + Random output layer (0 - 18)           
        self.fc2 = nn.Linear(64, 19) 

    def forward(self, x, rand_num):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        #x = x.view(-1, 10)
        x = torch.flatten(x,1)
        new_x = torch.cat((x, rand_num), dim=1)

        new_x = self.fc2(F.relu(self.fc1(new_x))) 

        return F.log_softmax(x, dim = 1), F.log_softmax(new_x, dim = 1)

#Taking Model from Network

In [22]:
model = Network()

#Training and Testing of a Model

In [23]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (image, label, randNum, sum_label) in enumerate(pbar):
        image, label, randNum,sum_label = image.to(device), label.to(device),randNum.to(device),sum_label.to(device)
        optimizer.zero_grad()
        output, sum_output = model(image,randNum )
        loss = F.nll_loss(output, label)
        sum_loss = F.nll_loss(sum_output,sum_label)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    sum_correct = 0
    with torch.no_grad():
        for image, label, randNum, sum_label in test_loader:
            image, label, randNum,sum_label = image.to(device), label.to(device),randNum.to(device),sum_label.to(device)
            output, sum_output = model(image, randNum)
            test_loss += F.nll_loss(output, label, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            sum_pred = sum_output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(label.view_as(pred)).sum().item()
            sum_correct += pred.eq(sum_label.view_as(sum_pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}, Sum_Accuracy: {}/{} ({:.0f}%)\n'.format(
     #   test_loss, correct, len(test_loader.dataset),
      #  100. * correct / len(test_loader.dataset),100. * sum_correct / len(test_loader.dataset)))
    
    print(f'{test_loader} set: Average loss: {test_loss:.3f}, MNIST Accuracy:{100. * correct/len(test_loader.dataset)}, Addition_Accuracy:{100. * sum_correct/len(test_loader.dataset)}')

In [25]:
model = Network().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.95)

for epoch in range(1, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.32161745429039 batch_id=468: 100%|██████████| 469/469 [00:29<00:00, 15.93it/s]


<torch.utils.data.dataloader.DataLoader object at 0x7f69288a3e20> set: Average loss: 0.321, MNIST Accuracy:86.83, Addition_Accuracy:9.0


loss=0.3808058202266693 batch_id=468: 100%|██████████| 469/469 [00:29<00:00, 15.93it/s]


<torch.utils.data.dataloader.DataLoader object at 0x7f69288a3e20> set: Average loss: 0.302, MNIST Accuracy:87.39, Addition_Accuracy:9.09


loss=0.4174554646015167 batch_id=468: 100%|██████████| 469/469 [00:29<00:00, 15.96it/s]


<torch.utils.data.dataloader.DataLoader object at 0x7f69288a3e20> set: Average loss: 0.296, MNIST Accuracy:87.62, Addition_Accuracy:8.7


loss=0.20930629968643188 batch_id=468: 100%|██████████| 469/469 [00:29<00:00, 16.06it/s]


<torch.utils.data.dataloader.DataLoader object at 0x7f69288a3e20> set: Average loss: 0.289, MNIST Accuracy:87.76, Addition_Accuracy:8.36
